Perform ETL 

In [11]:
from main import run_pipeline

run_pipeline()


INFO:root:Initializing Spotify API client...
INFO:root:Spotify API client initialized successfully.
INFO:root:Searching for playlist 'Top Hits of 2020'...
INFO:root:Searching for playlist 'Top Hits of 2021'...
INFO:root:Searching for playlist 'Top Hits of 2022'...
INFO:root:Searching for playlist 'Top Hits of 2023'...
INFO:root:Searching for playlist 'Top Hits of 2024'...
INFO:root:Fetching tracks for playlist 2020...
INFO:root:Fetching tracks for playlist 2021...
INFO:root:Fetching tracks for playlist 2022...
INFO:root:Fetching tracks for playlist 2023...
INFO:root:Fetching tracks for playlist 2024...
INFO:root:Transforming raw data into DataFrame


OSError: The data folder 'data' doesn't exist.

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/top_hits.csv")
df.head()



ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Corrélation entre artist_popularity, artist_followers, track_popularity
corr = df[["artist_followers", "artist_popularity", "track_popularity"]].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

# Scatter plot: followers vs popularité artiste
sns.scatterplot(
    data=df, x="artist_followers", y="artist_popularity", alpha=0.5
)
plt.title("Artist Popularity vs Followers")
plt.xscale("log")  # followers varient beaucoup
plt.show()

# Scatter plot: track_popularity vs artist_popularity
sns.scatterplot(
    data=df, x="track_popularity", y="artist_popularity", alpha=0.5
)
plt.title("Artist Popularity vs Track Popularity")
plt.show()
